In [1]:
#imports

%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import glob
import os
import time
import requests
import bs4
import json
from requests.auth import HTTPBasicAuth
from flatten_json import flatten

In [2]:
# 20 requests every 1 seconds(s)
# 100 requests every 2 minutes(s)

apikey = "RGAPI-d2088fc5-5f70-4703-8360-03eabf53dcca"

In [6]:
# get challenger summonerId from league-exp-api
# 1.2 delay bc short list

req = requests.get("https://euw1.api.riotgames.com/lol/league-exp/v4/entries/RANKED_SOLO_5x5/CHALLENGER/I?page=1&api_key={}"
                   .format(apikey),time.sleep(1.2))
assert req.status_code == 200
data = req.json()
sumids = [d['summonerId'] for d in data]

In [7]:
print(sumids[0])

xVfAvKB6MsfZ-ihvaeOpFFjCSWq8e9ba07jtBe4E0IYw5mY


In [8]:
# get accountId via summonerId via summoner-api
# 1.2 delay bc short list

accids = []
for sumid in sumids:
    sumreq = requests.get("https://euw1.api.riotgames.com/lol/summoner/v4/summoners/{}?api_key={}"
                          .format(sumid,apikey),time.sleep(1.2))
    assert sumreq.status_code == 200
    accids.append(sumreq.json()['accountId'])

In [9]:
print(accids[0])

0LKuAA-YP37tFAKtCSLfvyH8qSirbyYxXQeQH2HlIj0xWMg


In [10]:
# get game ids from match history via match-api
# 1.3 delay bc long list

gamids = []

for acc in accids:
    matlreq = requests.get("https://euw1.api.riotgames.com/lol/match/v4/matchlists/by-account/{}?api_key={}"
                           .format(acc,apikey),time.sleep(1.3))
    assert matlreq.status_code == 200
    gamids += [d['gameId'] for d in matlreq.json()['matches']]

In [11]:
# print original length, then correct overlap from shared games and print new length

print(len(gamids))

gamids = set(gamids)

print(len(gamids))

20500
11112


In [ ]:
# get individual match information from gamids list via match-api
# 1.5 delay bc long list and repeated issues with 

matchlist = []

for gam in gamids:
    gamreq = requests.get("https://euw1.api.riotgames.com/lol/match/v4/matches/{}?api_key={}"
                          .format(gam,apikey),time.sleep(1.3))
    
    i=0
    while gamreq.status_code != 200:
        gamreq = requests.get("https://euw1.api.riotgames.com/lol/match/v4/matches/{}?api_key={}"
                          .format(gam,apikey),time.sleep(1.3))
        i+=1
        if i == 5:
            break
            
    match = flatten(gamreq.json())
    matchlist.append(match)

In [13]:
gamreq.status_code

504

In [14]:
# check length of matchlist 

check = [len(m) for m in matchlist]
len(matchlist),max(check),min(check)

(176, 1550, 1212)

In [15]:
# create df from matches

df = pd.DataFrame(matchlist)

In [16]:
df

,gameId,platformId,gameCreation,gameDuration,queueId,mapId,seasonId,gameVersion,gameMode,gameType,...,participants_6_timeline_damageTakenDiffPerMinDeltas_30-end,participants_7_timeline_csDiffPerMinDeltas_30-end,participants_7_timeline_csDiffPerMinDeltas_20-30,participants_7_timeline_xpDiffPerMinDeltas_30-end,participants_7_timeline_xpDiffPerMinDeltas_20-30,participants_7_timeline_damageTakenDiffPerMinDeltas_30-end,participants_7_timeline_damageTakenDiffPerMinDeltas_20-30,participants_8_timeline_csDiffPerMinDeltas_30-end,participants_8_timeline_xpDiffPerMinDeltas_30-end,participants_8_timeline_damageTakenDiffPerMinDeltas_30-end
0,4681072641,EUW1,1593268978835,1299,420,11,13,10.13.326.4870,CLASSIC,MATCHED_GAME,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,4707844099,EUW1,1594758284675,1438,420,11,13,10.14.327.8505,CLASSIC,MATCHED_GAME,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4679991311,EUW1,1593201304458,1826,420,11,13,10.13.326.4870,CLASSIC,MATCHED_GAME,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4648730648,EUW1,1591566971144,1523,420,11,13,10.11.322.2991,CLASSIC,MATCHED_GAME,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4632641560,EUW1,1590840449379,1908,420,11,13,10.11.322.2991,CLASSIC,MATCHED_GAME,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
171,4695458377,EUW1,1594079247622,1948,400,11,13,10.13.326.4870,CLASSIC,MATCHED_GAME,...,NaN,NaN,3.8,NaN,-254.2,NaN,391.00,NaN,NaN,NaN
172,4617306699,EUW1,1590182358342,1350,420,11,13,10.10.320.3039,CLASSIC,MATCHED_GAME,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
173,4697260629,EUW1,1594184790037,1505,450,12,13,10.14.327.1684,ARAM,MATCHED_GAME,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
174,4725310037,EUW1,1595666342665,2323,420,11,13,10.15.328.8759,CLASSIC,MATCHED_GAME,...,310.4,1.4,0.4,274.7,-110.2,310.4,-179.35,-0.4,-0.4,-795.2


In [ ]:
challgames = list(df['gameId'])
with open('euw_chall_gam_id_list.txt', 'w') as f:
    for item in gmgames:
        f.write("%s\n" % item)

In [ ]:
compression_opts = dict(method='zip',
                        archive_name='euchall.csv')  
df.to_csv('euchall.zip', index=False,
          compression=compression_opts)